<h1>SI 564 Final Project: Trails in U.S. National Parks</h1>
<p>Haley Johnson</p>

In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine 
from IPython.display import Image

<h2>Database Diagram</h2>
<p><a href = "https://www.kaggle.com/datasets/planejane/national-park-trails">National Parks data</a></p>

In [2]:
Image(url="nat_parks_erd.png", width=800, height=800)

<h2>Split Data Into Tables</h2>

In [3]:
df = pd.read_csv("trails_data.csv")

In [4]:
df = df.drop(columns = ['trail_id', 'city_name', 'country_name', '_geoloc', 'features', 'activities'])

In [ ]:
df['area_name'] = df['area_name'].str.replace(" National Park", "")

In [14]:
def create_table(df, col):
    temp = df[col].unique()
    df = pd.DataFrame(temp, columns = [col])
    df = df.reset_index()
    df = df.rename(columns = {'index': 'id'})
    return df
    

In [17]:
parks_df = create_table(df, 'area_name')
states_df = create_table(df, 'state_name')
routes_df = create_table(df, 'route_type')

In [31]:
# METRICS ROWS
# elevation gain in METERS
# length in METERS

# IMPERIAL ROWS
# elevation gain in FEET
# length in YARDS / meters ?? 

<h3>Trails Table</h3>

In [32]:
metric = df[df['units'] == 'm']
imperial = df[df['units'] == 'i']

In [33]:
def meters_to_yards(s):
    '''
    Takes in column of dataframe 
    
    Converts metric units to imperial ones
    '''
    return s * 1.09361
    

In [34]:
metric['elevation_gain'] = metric['elevation_gain'].apply(meters_to_yards)
metric['length'] = metric['length'].apply(meters_to_yards)

<ipython-input-34-5803d4a6adb9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metric['elevation_gain'] = metric['elevation_gain'].apply(meters_to_yards)
<ipython-input-34-5803d4a6adb9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metric['length'] = metric['length'].apply(meters_to_yards)


In [38]:
df = pd.concat([imperial, metric])

<h3>Parks Table</h3>

In [20]:
parks_df['area_name'] = parks_df['area_name'].str.replace(" National Park", "")

<h3>States Table</h3>

<h3>Route Types Table</h3>

<h2>Normalize</h2>

<h2>Write To SQL</h2>

In [ ]:
df.to_sql("table",con=engine)